In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import os

In [47]:
DATASET_PATH = "/Users/victor/programming/random/zapcheck/Phishing_Email.csv"

df = pl.read_csv(DATASET_PATH)

df = df.select(
  pl.col("Email Text").alias("texto"),
  pl.when(pl.col("Email Type") == "Phishing Email") 
    .then(pl.lit(1))
    .when(pl.col("Email Type") == "Safe Email")
    .then(pl.lit(0))
    .alias("label") 
)

#schema = {
#  "texto": pl.String,
#  "label": pl.Int8
#}

#df = pl.DataFrame(schema=schema)

#for file in os.listdir(os.path.join(DATASET_PATH, "full_texts/fake")):
#  with open(os.path.join(DATASET_PATH, "full_texts/fake", file), "r") as f:
#    df = df.vstack(pl.DataFrame({"texto": [f.read()], "label": [1]}, schema=schema))

#for file in os.listdir(os.path.join(DATASET_PATH, "full_texts/true")):
#  with open(os.path.join(DATASET_PATH, "full_texts/true", file), "r") as f:
#    df = df.vstack(pl.DataFrame({"texto": [f.read()], "label": [0]}, schema=schema))

In [37]:
df

texto,label
str,i64
"""A universidade pode fazer isso…",1
"""Filho sobre exoneração de alim…",0
"""Comprei um Monitor, veio quebr…",1
"""Venda casada na abertura da co…",1
"""O que significa uma ou mais em…",1
…,…
"""A Vivo me enrolando""",1
"""Minha tia trabalhou durante 5 …",0
"""Quero fazer a mudança do meu n…",0


In [38]:
df.filter(df["label"] == 0).head()

texto,label
str,i64
"""Filho sobre exoneração de alim…",0
"""Tempo de pagamento aviso prévi…",0
"""Como garantir autoria de uma o…",0
"""Desabafo de um clínico CLT""",0
"""Cachorro da vizinha não para d…",0


In [35]:
print("Dataset size: ", len(df))
print("phishing: ", len(df.filter(df["label"] == 0)))
print("emais normais: ", len(df.filter(df["label"] == 1)))


Dataset size:  150
phishing:  75
emais normais:  75


In [71]:
print("Duplicados: ", df.is_duplicated().sum())

print("Nulos: ", df.null_count())

df = df.unique(maintain_order=True)

Duplicados:  1588
Nulos:  shape: (1, 2)
┌───────┬───────┐
│ texto ┆ label │
│ ---   ┆ ---   │
│ u32   ┆ u32   │
╞═══════╪═══════╡
│ 16    ┆ 0     │
└───────┴───────┘


In [73]:
import spacy
import unicodedata

#uv run spacy download pt_core_news_sm

def remove_acentos(input_str) -> str:
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.category(c) == 'Mn'])

def remove_numeros_pequenos(text):
    import re
    return re.sub(r'\b\d{1,3}\b', '', text)

df = df.with_columns(
    pl.col("texto")
    .str.to_lowercase()
    .str.replace_all(r'http?://\S+', '') #remove links
    .str.replace_all(r'[^a-zA-Z0-9\s]', '') #remove caracteres especiais
    .map_elements(remove_acentos, return_dtype=pl.Utf8)
    .str.replace_all(r'\s+', ' ')
    .map_elements(remove_numeros_pequenos, return_dtype=pl.Utf8)
    .str.slice(0, 1500000)
    .alias("texto")
)
#remove as stopwords

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1500000

def remove_stopwords(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if not token.is_stop])

df = df.with_columns(
  texto=pl.col("texto").map_elements(remove_stopwords, return_dtype=pl.Utf8)
)

df = df.drop_nulls()


In [74]:
df.write_csv("./dados_tratados.csv")